In [14]:
import pandas as pd
import numpy as np
from sklearn import svm, datasets

In [3]:
iris = datasets.load_iris()

In [4]:
df = pd.DataFrame(iris.data,columns = iris.feature_names)
df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2


In [5]:
df['flower'] = iris.target
# df['flwr'] = iris.target
df['flower'] = df['flower'].apply(lambda x:iris.target_names[x])
# df['flwr'] = df['flwr'].map(dict(enumerate(iris.target_names)))
# df.drop(['flwr'],axis=1)
df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),flower
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


In [6]:
from sklearn.model_selection import train_test_split
x_train , x_test , y_train, y_test = train_test_split(iris.data, iris.target, test_size=0.3)

In [7]:
model = svm.SVC(kernel='rbf',C=30, gamma='auto')
model.fit(x_train,y_train)
model.score(x_test,y_test)

0.9777777777777777

In [8]:
from sklearn.model_selection import cross_val_score
cross_val_score(svm.SVC(kernel='linear',C=10,gamma='auto'),iris.data , iris.target , cv=5)

array([1.        , 1.        , 0.9       , 0.96666667, 1.        ])

In [9]:
cross_val_score(svm.SVC(kernel='rbf',C=10,gamma='auto'),iris.data , iris.target , cv=5)

array([0.96666667, 1.        , 0.96666667, 0.96666667, 1.        ])

In [10]:
cross_val_score(svm.SVC(kernel='rbf',C=20,gamma='auto'),iris.data , iris.target , cv=5)

array([0.96666667, 1.        , 0.9       , 0.96666667, 1.        ])

In [21]:
kernels= ['rbf','linear']
C= [1,10,20]
avg_scores = {}
for kval in kernels:
    for cval in C:
        cv_scores = cross_val_score(svm.SVC(kernel= kval , C= cval, gamma= 'auto'),iris.data, iris.target, cv=5)
        avg_scores[kval + '_' + str(cval)] = np.average(cv_scores)
avg_scores

{'rbf_1': 0.9800000000000001,
 'rbf_10': 0.9800000000000001,
 'rbf_20': 0.9666666666666668,
 'linear_1': 0.9800000000000001,
 'linear_10': 0.9733333333333334,
 'linear_20': 0.9666666666666666}

In [16]:
from sklearn.model_selection import GridSearchCV
clf = GridSearchCV(svm.SVC(gamma='auto'),{
    'C' :[1,10,20] ,
    'kernel': ['rbf','linear']
},cv =5 ,return_train_score= False )
clf.fit(iris.data, iris.target)
clf.cv_results_



{'mean_fit_time': array([0.00325899, 0.00294871, 0.00311112, 0.00316768, 0.        ,
        0.00300136]),
 'std_fit_time': array([0.00329463, 0.00309192, 0.00148768, 0.00388786, 0.        ,
        0.00314104]),
 'mean_score_time': array([0.00207915, 0.00080276, 0.00133185, 0.        , 0.0016499 ,
        0.00117817]),
 'std_score_time': array([0.00324962, 0.00098317, 0.00111226, 0.        , 0.00329981,
        0.00096316]),
 'param_C': masked_array(data=[1, 1, 10, 10, 20, 20],
              mask=[False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_kernel': masked_array(data=['rbf', 'linear', 'rbf', 'linear', 'rbf', 'linear'],
              mask=[False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'C': 1, 'kernel': 'rbf'},
  {'C': 1, 'kernel': 'linear'},
  {'C': 10, 'kernel': 'rbf'},
  {'C': 10, 'kernel': 'linear'},
  {'C': 20, 'kernel': 'rbf'},
  {'C': 20, 'kernel': 'linear'}],


In [18]:
df = pd.DataFrame(clf.cv_results_)
# df
df[['param_C','param_kernel','mean_test_score']]

,param_C,param_kernel,mean_test_score
0,1,rbf,0.980000
1,1,linear,0.980000
2,10,rbf,0.980000
3,10,linear,0.973333
4,20,rbf,0.966667
5,20,linear,0.966667


In [22]:
clf.best_score_

0.9800000000000001

In [24]:
clf.best_params_

{'C': 1, 'kernel': 'rbf'}

In [28]:
from sklearn.model_selection import RandomizedSearchCV
rs = RandomizedSearchCV(svm.SVC(gamma='auto'),{
    'C':[1,10,20],
    'kernel' :['rbf','linear']
}, cv=5, return_train_score=False, n_iter=2)
rs.fit(iris.data, iris.target)
pd.DataFrame(rs.cv_results_)[['param_C','param_kernel','mean_test_score']]

,param_C,param_kernel,mean_test_score
0,1,rbf,0.980000
1,10,linear,0.973333


In [29]:
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

In [35]:
model_params = {
    'svm': {
        'model': svm.SVC(gamma='auto'),
        'params': {
            'C':[1,10,20],
            'kernel': ['rbf','linear']
        }
    },
    'random_forest': {
        'model': RandomForestClassifier(),
        'params':{
            'n_estimators': [1,5,10]
        }
    },
    'logistic_resgression': {
        'model': LogisticRegression(solver='liblinear',multi_class='auto'),
        'params':{
            'C':[1,10,20]
        }
    }
}



In [37]:
scores = []
for model_names, mp in model_params.items():
    clf = GridSearchCV(mp['model'], mp['params'], cv=5, return_train_score=False)
    clf.fit(iris.data, iris.target)
    scores.append({
        'model': model_names, 
        'best_score': clf.best_score_,
        'best_params': clf.best_params_
    })
df = pd.DataFrame(scores , columns=['model','best_score' ,'best_params' ])
df

,model,best_score,best_params
0,svm,0.980000,"{'C': 1, 'kernel': 'rbf'}"
1,random_forest,0.960000,{'n_estimators': 1}
2,logistic_resgression,0.966667,{'C': 10}
